<a href="https://colab.research.google.com/github/Location-Artistry/GEO-DEV-NOTEBOOKS/blob/main/AGOL_PYTHON_EXPORT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Working Water Data Pandas AWQMS Export**
Updated August 12th - Working with pandas dataframe exporting to excel   
Working function to run download and CSV formatting in pandas   
**8-14 Completed: Download & Export Functions, QuickMap Display**
**8-25 delMultiple, updated Kzoo Dashboard Status**   
**8-25 This Notebook has most current Python Functions!!!**

# Install and Import Libraries
**RUN FIRST**

In [ ]:
!pip install xlsxwriter
!pip install xlrd
!pip install openpyxl
!pip install arcgis
#!pip install pdfkit
#import pdfkit
import pandas as pd
from pathlib import Path
import sys
import os
import datetime as dt
import time as tm
import math
from arcgis.gis import GIS, Item
from arcgis.env import active_gis
from arcgis.features import FeatureLayerCollection
from arcgis.mapping import WebMap
from IPython.display import display
import getpass
from pathlib import Path
from zipfile import ZipFile
from openpyxl import load_workbook

In [ ]:
userLogin()

In [ ]:
import json 
import requests
import arcgis.mapping
jsonURL = 'https://services5.arcgis.com/UDWrEU6HdWNYIRIV/arcgis/rest/services/WATER_SAMPLING_2020_d4e27d13e2da43bfb8e300db323e349d/FeatureServer?f=json'
# print(jsonURL)
response = requests.get(jsonURL)
resText = response.text
data = json.loads(resText)
#display(data)

arcgis.mapping.export_map(data,'JPG')

In [ ]:
#searchInfo = searchByKeywords(gis,'')
for x in searchInfo[3]:
  print(f'{x} - {searchInfo[3][x]}')
#getUserContent(gis)
#searchInfo[3].url

# Python Functions from my ArcGIS Python Colab Notebook  
**MUST RUN THIS CELL AFTER INSTALL AND IMPORT THEN - userLogin() -**


In [ ]:
# Collection of all ArcGIS Python API Helper Functions
# user login functions, ask if user would like additional logins
def userLogin():
    userID = input(f'ArcGIS Online USER ID: ')
    passWord = getpass.getpass('PASSWORD: ')
    try:
        global gis
        gis = GIS("https://www.arcgis.com", userID, passWord)
        print(f'SUCCESS - CONNECTED TO: {gis.users.me.username} ACCOUNT as <gis>')
        print(gis)
        addUsers = input(f'Additional User Login(YES/NO)? ')
        if addUsers.upper() == 'YES':
            additionalUserLogin()
        else:
            print(f'YOU MAY NOW PROCEED...')
    except:
        print(f'ERROR DID NOT CONNECT TO: {userID}')

def additionalUserLogin():
    userID = input(f'ArcGIS Online USER ID: ')
    passWord = getpass.getpass('PASSWORD: ')
    try:
        global gis2
        gis2 = GIS("https://www.arcgis.com", userID, passWord)
        print(f'SUCCESS - CONNECTED TO: {gis2.users.me.username} ACCOUNT as <gis2>')
        print(gis2)
    except:
        print(f'ERROR DID NOT CONNECT TO: {userID}')

# get list of all owner AGOL items, print list with title, id, type, and categories
def getUserContent(gisInfo):
    try:
        my_content = gisInfo.content.search(query="owner:" + gisInfo.users.me.username, item_type="", max_items=200)
        for x in my_content:
            strMod = str(x.modified)
            stampInt = int(strMod[0:10])
            print(f'{x.title} - {x.id} - {x.type} - {x.categories} - {dt.datetime.fromtimestamp(stampInt)}')
    except:
        print('ERROR could not get user content')

# Clone item using id of item passed to function
def cloneItem(gisInfo, gisInfo2, cloneID):    
    try:
        itemToClone = gisInfo.content.get(cloneID)
        print('Cloning:' + itemToClone.title + ' - ' + itemToClone.id + ' -',itemToClone.type)
        clonedItem = gisInfo2.content.clone_items(items=[itemToClone])
        print(f'Cloned Item: {clonedItem[0]}')
        #return clonedItem
    except:
        print('ERROR Could Not Clone')

# updated searchByKeywords, returns LIST of items 8-14-2020
def searchByKeywords(gisInfo, searchKeywords):
    try:
        searchContent = gisInfo.content.search(query=f'{searchKeywords}', item_type='', max_items=50)
        x = 0
        for z in searchContent:
          strMod = str(z.modified)
          stampInt = int(strMod[0:10])
          print(f'{x} - {z.title} - {z.id} - {z.type} - {z.categories} - {dt.datetime.fromtimestamp(stampInt)}')
          x += 1
        return searchContent
    except:
        print('ERROR Search not Successful')

# find item by keywords and display visual card
def searchByKeyViz(gisInfo, searchKeywords):
    try:
        searchContent = gisInfo.content.search(query=f'{searchKeywords}', item_type='', max_items=50)
        for z in searchContent:
            print(f'title: {z.title} - itemID: {z.id} - type: {z.type}')
            display(z)
    except:
        print('ERROR Search not Successful')
        
# return all keys and values for item when passed itemID string
def getItemKeysValues(gisInfo, idString):
    try:
        getFeature = gisInfo.content.get(idString)
        for key, value in getFeature.items():
            print(key,': ', value)
    except:
        print('ERROR GET Keys/Values not Successful')
        
# takes itemID and gets and returns layerObject if exist, otherwise 'no layers found'
def getLayers(gisInfo, idString):
    getFeature = gisInfo.content.get(idString)
    try:
        featureLayers = getFeature.layers
        z = 0 
        for x in featureLayers:
            print(f'Layer {z}: {x}')
            z += 1
    except:
        print('no layers found')
    return featureLayers

# supply feature layer itemID, and the layer number to display table head 
def getLayerTable(gisInfo, idString, layerNum):
    try:
        layerOutput = getLayers(gisInfo, idString)
        queryLayer = layerOutput[layerNum].query()
        display(queryLayer.sdf.head())
    except:
        print('ERROR no Layers Found')
        
# delete item by itemID
def deleteItem(gisInfo, idString):
    itemToDelete = gisInfo.content.get(idString)
    display(itemToDelete)
    delQuest = input(f'Are you sure you want to delete: {itemToDelete.title}')
    try:
        if delQuest.upper() == 'YES':
            print(f'DELETING: {itemToDelete.title}')
            itemToDelete.delete()
        else:
            print(f'NOT DELETING: {itemToDelete.title}')
    except:
        print(f'ERROR failed to DELETE: {itemToDelete.title}')

# delete multiple items by searchByKeywords() returned LIST 8-14-2020
def delMultiple(gisInfo, itemList):
  try:
    print('List of Items to be Deleted: ')
    for z in itemList:
          strMod = str(z.modified)
          stampInt = int(strMod[0:10])
          print(f'{z.title} - {z.id} - {z.type} - {z.categories} - {dt.datetime.fromtimestamp(stampInt)}')
    delQuest = input(f'SURE YOU WANT TO DELETE THESE?!?!?')
    if delQuest.upper() == 'YES':
      for z in itemList:
          print(f'DELETING {z.title}')
          itemToDelete = gisInfo.content.get(z.id)
          itemToDelete.delete()
      print('<FINISHED DELETION PROCESS>')
    else:
      print(f'NOT DELETING!')
  except:
      print(f'ERROR failed to DELETE: {itemToDelete.title}')

# List all user Dashboards and Dashboard Webmmaps
def ListAllDashWebmaps(gisInfo):
  source_admin_inventory = get_user_items(gisInfo, gisInfo.users.me)
  x = 0
  try:
    for dashboard in source_admin_inventory['Dashboard']:
        print(x, dashboard)
        dashWebmap = get_dash_wm(gisInfo, dashboard)
        print(dashWebmap)
        x += 1
  except:
    print("ERROR COULD NOT LIST DASHBOARDS") 

# generic function update targetLayer Features based on Table Records
def updateLayFeatFromTable(gisInfo, targetLayerID, matchAttrib, targetAttrib, sourceAttrib):
  try:
    getLayers = gisInfo.content.get(targetLayerID)
    targetLayer = getLayers.layers
    layerFeatures = targetLayer[0].query()
    sourceTable = getLayers.tables
    tableFeatures = sourceTable[0].query()
    for tableFeature in tableFeatures:
      tableFeatureID = tableFeature.attributes[matchAttrib]
      for layerFeature in layerFeatures:
        layerFeatureID = layerFeature.attributes[matchAttrib]
        if tableFeatureID == layerFeatureID:
          targetValue = tableFeature.attributes[sourceAttrib]
          layerFeature.set_value(targetAttrib, targetValue)
          print(f'feature: {layerFeatureID} from tableFeature: {tableFeatureID} set {targetAttrib} as: {targetValue}')
    layerEdits = targetLayer[0].edit_features(updates=layerFeatures)
    editCounter = 0
    for edits in layerEdits['updateResults']:
      editCounter+=1
    print(f'updated {getLayers.title} with {editCounter} edits from {getLayers.tables[0]}')
  except:
    print(f'update features failed for {getLayers.title}')
  
# generic function update targetLayer Features based on Table Record, adds break list for parameter categories mapping/analysis
# 8-14 Updated to screen for sampling records with blank values: 'None'
def updateLayFeatFromTableBreaks(gisInfo, targetLayerID, matchAttrib, targetAttrib, sourceAttrib, breaksList):
  try:
    getLayers = gisInfo.content.get(targetLayerID)
    targetLayer = getLayers.layers
    layerFeatures = targetLayer[0].query()
    sourceTable = getLayers.tables
    tableFeatures = sourceTable[0].query()
    for tableFeature in tableFeatures:
      #tableFeatureID = tableFeature.attributes[matchAttrib]
      print(tableFeature.attributes['WATER_TEMP'] is None)
      if (tableFeature.attributes['WATER_TEMP'] is None) != True:
        tableFeatureID = tableFeature.attributes[matchAttrib]
        for layerFeature in layerFeatures:
          layerFeatureID = layerFeature.attributes[matchAttrib]
          if tableFeatureID == layerFeatureID:
            targetValue = tableFeature.attributes[sourceAttrib]
            x = 1
            for breakVal in breaksList:
              if targetValue > breakVal:
                print('none')
              else:
                layerFeature.set_value(targetAttrib, x)
                print(f'feature: {layerFeatureID} from tableFeature: {tableFeatureID} set {targetAttrib}: {targetValue} as: {x}')
                break
              x+=1
    layerEdits = targetLayer[0].edit_features(updates=layerFeatures)
    editCounter = 0
    for edits in layerEdits['updateResults']:
      editCounter+=1
    # hide REST infor for updated layers and tables
    # print(f'updated {getLayers.title} with {editCounter} edits from {getLayers.tables[0]}')
    print(f'updated {getLayers.title} with {editCounter} edits')
  except:
    print(f'update features failed for {getLayers.title}')

# download Feature Layer data from AGOL, unzip contents to folder with item.title name
# Export Formats: Shapefile | CSV | File Geodatabase | Feature Collection | GeoJson | Scene Package | KML | Excel
def downloadItem(gisInfo, idString):
    try:
        downloadData = gisInfo.content.get(idString)
        dataPath = Path('/data')
        print(f'Downloading: {downloadData.title} to {dataPath} directory')
        if not dataPath.exists():
          dataPath.mkdir()
        # this portion for feature service
        downloadExport = downloadData.export(title=downloadData.title, export_format="CSV")
        zipPath = downloadExport.download(save_path=dataPath)
        # preparing to extract files to directory with item.title name
        #zipPath = downloadData.download(save_path=dataPath)
        extractPath = dataPath.joinpath(downloadData.title)
        # extract files to /data directory
        zipFiles = ZipFile(zipPath)
        zipFiles.extractall(path=extractPath)
        print(f'list of Files extracted to: {extractPath}')
        print(list(file.name for file in extractPath.glob('*')))
    except:
        print('ERROR DOWNLOAD did not workings!')

def searchItem(gisInfo, searchKeywords, itemType):
    try:
        searchContent = gisInfo.content.search(query=f'{searchKeywords}', item_type=itemType, max_items=25)
        if itemType == 'Feature Service':
            x = 0
            print(f'<Search Query for {searchKeywords}>')
            for z in searchContent:
                strMod = str(z.modified)
                stampInt = int(strMod[0:10])
                print(f'{x} - {z.title} - {z.id} - {z.type} - {z.categories} - {dt.datetime.fromtimestamp(stampInt)}')
                x += 1
            layInd = int(input(f'Index of selected Feature Layer: '))
            addLayer = gisInfo.content.get(searchContent[layInd].id)
            return addLayer
        elif itemType == 'Web Map':
            x = 0
            print(f'<Search Query for {searchKeywords}>')
            for z in searchContent:
                strMod = str(z.modified)
                stampInt = int(strMod[0:10])
                print(f'{x} - {z.title} - {z.id} - {z.type} - {z.categories} - {dt.datetime.fromtimestamp(stampInt)}')
                x += 1
            layInd = int(input(f'Index of selected Feature Layer: ')) or 'NONE'
            #print(searchContent[layInd])
            mapReturn = searchContent[layInd]
            return mapReturn
    except:
        print('ERROR Search not Successful')
        
def quickMap():
  mapType = input(f'(YES) for QuickMap (NO) for Existing: ')
  if mapType.upper() == 'NO':
    mapSize = ['480px','720px','960px']
    print(f'<You entered {mapType} please login below>')
    userLogin() 
    mapKeywords = input(f'Name of WebMap to Search for: ') or ''
    mapObj = searchItem(gis,mapKeywords,'Web Map')
    map = gis.map(mapObj)
    sizeIn = int(input(f'MAP SIZE (0)SMALL (1)MEDIUM (2)HUGE: '))
    map.layout.height = mapSize[sizeIn]
    display(map)
  else:
    print(f'<You entered {mapType} Opening QuickMap>')
    mapList = ['topo','hybrid','streets','dark-gray','terrain']
    mapDimen = ['2D','3D']
    mapSize = ['480px','720px','960px']
    mapLoc = input(f'Location (default=Michigan): ') or 'Michigan'
    mapBaseNum = input(f'Basemap (default=topo (1=hybrid,2=streets,3=dark-gray,4=terrain): ') or 0
    mapDimIn = input(f'ENTER (1) for 3D Map: ') or 0
    atlasLayers = input(f'Layers from Living Atlas(Enter for None): ') or 'NONE'
    gisNone = GIS()
    map = gisNone.map(mapLoc)
    map.basemap = mapList[int(mapBaseNum)]
    if atlasLayers != 'NONE':
        layerDisplay = searchItem(gisNone, atlasLayers,'Feature Service')
        for layrs in layerDisplay.layers:
            map.add_layer(layrs)
    map.mode = mapDimen[int(mapDimIn)]
    sizeIn = int(input(f'MAP SIZE (0)SMALL (1)MEDIUM (2)HUGE: '))
    map.layout.height = mapSize[sizeIn]
    display(map)

# Delete all features from selected Feature Service, may need more debugging
def delAllFeatures(gisInfo, idString):
    delFeatures = gisInfo.content.get(idString)
    display(delFeatures)
    delQuest = input(f'Are you sure you want to delete all the feature in? {delFeatures.title}')
    try:
        if delQuest.upper() == 'YES':
            print('in loop')
            print(f'DELETING ALL FEATURES IN: {delFeatures.title}')
            featDelRes = []
            targetLayer = delFeatures.layers
            layerFeatures = targetLayer[0].query()
            for feature in layerFeatures:
              #print(f'features: {feature.attributes}')
              featDelRes.append(targetLayer[0].edit_features(deletes=str(feature.attributes['objectid'])))
            return featDelRes
        else:
            print(f'NOT DELETING: {delFeatures.title}')
    except:
        print(f'ERROR failed to DELETE: {delFeatures.title}')

# *******ESRI pre-made helper functions********
def is_hosted(gisInfo, item):
    return [keyword for keyword in item.typeKeywords if "Hosted" in keyword] 

# Prints all layers in a webmap, very handy
def print_webmap_inventory(gisInfo, wm):
    wm_obj = WebMap(wm)
    print(f"{wm_obj.item.title}\n{'-'*100}")
    for wm_layer in wm_obj.layers:
        try:
            if is_hosted(Item(gisInfo, wm_layer['itemId'])):
                print(f"{' '*2}{wm_layer['title']:40}HOSTED{' ':5}"
                      f"{wm_layer['layerType']:20}{dict(wm_layer)['itemId']}")
            else:
                print(f"{' '*2}{wm_layer['title']:40}other{' ':6}"
                      f"{wm_layer['layerType']:20}{wm_layer.id}") 
        except:
            print(f"{' '*2}{wm_layer['title']:40}other{' ':6}"
                  f"{wm_layer['layerType']:20}{wm_layer.id}")
    print("\n")

def get_webmap_list(wm):
    wm_obj = WebMap(wm)
    wmList = []
    print(f"{wm_obj.item.title}\n{'-'*100}")
    for wm_layer in wm_obj.layers:
        # print(wm_layer.itemId)
        wmList.append(wm_layer.itemId)
    return(wmList)
    
def displayWebmapLayers(gisInfo, idList):
    for id in idList:
        displayLayer = gisInfo.content.get(id)
        display(displayLayer)

def get_user_items(gisInfo, user):
    user_inventory = {}
    user_items = gisInfo.content.search(query=f"* AND owner:{user.username}", 
                                           max_items=500)
    for item in user_items:
        if item.type not in user_inventory:
            user_inventory[item.type] = [i 
                                         for i in user_items 
                                         if i.type == item.type]
    return user_inventory

def print_user_inventory(inventory):
    for itype, ilist in inventory.items():
        try:
            print(f"{itype}\n{'-'*50}")
            for i in ilist:
                print(f"{' ':3}{i.title:50}")
            print("\n")
        except Exception as e:
            print(f"\t\tOperation failed on: {i.title}")
            print(f"\t\tException: {sys.exc_info()[1]}")
            continue
            
def get_dash_wm(gisInfo, dash):
    return [gisInfo.content.get(widget['itemId']) 
            for widget in dash.get_data()['widgets'] 
            if widget['type'] == "mapWidget"]

# Python API Function List  
**userLogin()** - **additionalUserLogin()** - **getUserContent** *(gisInfo)* - **cloneItem** *(gisInfo, gisInfo2, cloneID)*  
**searchByKeywords** *(gisInfo, searchKeywords)* - **searchByKeyViz** *(gisInfo, searchKeywords)*  
**getItemKeysValues** *(gisInfo, idString)* - **getLayers** *(gisInfo, idString)*
**getLayerTable** *(gisInfo, idString, layerNum)* - **deleteItem** *(gisInfo, idString)*  
**ListAllDashWebmaps** *(gisInfo)* - **updateLayFeatFromTable** *(gisInfo, targetLayerID, matchAttrib, targetAttrib, sourceAttrib)*  
**updateLayFeatFromTableBreaks** *(gisInfo, targetLayerID, matchAttrib, targetAttrib, sourceAttrib, breaksList)*  
**downloadItem** *(gisInfo, idString)* - **searchItem** *(gisInfo, searchKeywords, itemType)* - **quickMap()**   
**delAllFeatures** *(gisInfo, itemID)* - **delMultiple** *(gisInfo, itemList)*


---


# ESRI pre-made helper functions  
**is_hosted** *(gisInfo, item)* - **print_webmap_inventory** *(gisInfo, wm)* - **get_webmap_list** *(wm)*  
**displayWebmapLayers** *(gisInfo, idList)* - **get_user_items** *(gisInfo, user)*  
**print_user_inventory** *(inventory)* - **get_dash_wm** *(gisInfo, dash)*


# Work in Progress Cell(s)

# Water Quality Dashboard Display Update Functions
Updated KZOO Turb Status 7-23-2020 3:05 pm   
Updated NHBP DO Status 7-23-2020 3:08 pm
Upated NHBP DO Status 8-12-2020    
Updated KZOO Turb Status 8-15-2020 - Refactored Update Layer function, check for 'None'

In [ ]:
# generic function update targetLayer Features based on Table Record, adds break list for parameter categories mapping/analysis
# 8-14 Updated to screen for sampling records with blank values: 'None'
def updateLayFeatFromTableBreaks(gisInfo, targetLayerID, matchAttrib, targetAttrib, sourceAttrib, breaksList):
  try:
    getLayers = gisInfo.content.get(targetLayerID)
    targetLayer = getLayers.layers
    layerFeatures = targetLayer[0].query()
    sourceTable = getLayers.tables
    tableFeatures = sourceTable[0].query()
    for tableFeature in tableFeatures:
      #tableFeatureID = tableFeature.attributes[matchAttrib]
      print(tableFeature.attributes['WATER_TEMP'] is None)
      if (tableFeature.attributes['WATER_TEMP'] is None) != True:
        tableFeatureID = tableFeature.attributes[matchAttrib]
        for layerFeature in layerFeatures:
          layerFeatureID = layerFeature.attributes[matchAttrib]
          if tableFeatureID == layerFeatureID:
            targetValue = tableFeature.attributes[sourceAttrib]
            x = 1
            for breakVal in breaksList:
              if targetValue > breakVal:
                print('none')
              else:
                layerFeature.set_value(targetAttrib, x)
                print(f'feature: {layerFeatureID} from tableFeature: {tableFeatureID} set {targetAttrib}: {targetValue} as: {x}')
                break
              x+=1
    layerEdits = targetLayer[0].edit_features(updates=layerFeatures)
    editCounter = 0
    for edits in layerEdits['updateResults']:
      editCounter+=1
    # hide REST infor for updated layers and tables
    # print(f'updated {getLayers.title} with {editCounter} edits from {getLayers.tables[0]}')
    print(f'updated {getLayers.title} with {editCounter} edits')
  except:
    print(f'update features failed for {getLayers.title}')

#userLogin()
kzooLayerNHBP = 'dff379381a6b4b73a1d80b9fd42784a8'
kzooLayerLA = '6b62d8b710e64b8abc79015fd7231b87'
waterSampNHBP = '680016d676e746f98743f51d28abac60'
match = 'SITE_ID'
turbTarget = 'TURB_STATUS_'
turbSource = 'TURB_NTU'
turbRefList = [3.93,10,40,1000]
DOtarget = 'DO_STATUS_'
DOsource = 'DO_mgl'
DOrefList = [5,6,7,100]
# calc Turbidity Status from most recent sample
#updateLayFeatFromTableBreaks(gis, kzooLayerNHBP, match, turbTarget, turbSource, turbRefList)
# calc DO Status from most recent sample
updateLayFeatFromTableBreaks(gis, waterSampNHBP, match, DOtarget, DOsource, DOrefList)

# **Working function to download and process WATER DATA**

1.   (COMP) Download and extract WATER_SAMPLING_2020 (itemID: '680016d676e746f98743f51d28abac60')  
2.   (COMP) Read Water Data Sample Table as Pandas Dataframe  
3. (COMP)Drop uneeded columns/attributes  
4. (NEXT STEP) Format correctly for AWQMS upload
5. Write as XLS for export to AWQMS


In [ ]:
# downloadReturn and exportAGOLdata work together with specified itemID to 
# package Export from AGOL into CSV, download and extract files, pass file
# path back to export func which creates pandas dataframe, 
# ALL WORKING 9-9 -> Finishing Refactoring...
# 9-10 -> AQWMS issues, fixed dates and times columns, more robust
def exportWaterData():
  
  def downloadReturn(gisInfo, idString):
    try:
        downloadData = gisInfo.content.get(idString)
        dataPath = Path('/data')
        print(f'Downloading: {downloadData.title} to {dataPath} directory')
        if not dataPath.exists():
          dataPath.mkdir()
        # this portion for feature service
        downloadExport = downloadData.export(title=downloadData.title, export_format="CSV")
        zipPath = downloadExport.download(save_path=dataPath)
        # preparing to extract files to directory with item.title name
        extractPath = dataPath.joinpath(downloadData.title)
        # extract files to /data directory
        zipFiles = ZipFile(zipPath)
        zipFiles.extractall(path=extractPath)
        # delete CSV created by export
        print(f'Deleting CSV export generated by Download: {downloadExport.title} ID: {downloadExport.id}')
        itemToDelete = gisInfo.content.get(downloadExport.id)
        itemToDelete.delete()
        print(f'list of Files extracted to: {extractPath}')
        print(list(file.name for file in extractPath.glob('*')))
        returnArray = []
        for x in extractPath.glob('*'):
          returnArray.append(x)
        return returnArray
    except:
        print('ERROR DOWNLOAD did not workings!')

  def exportDataFrame(df, exportName):
    try:
      writer = pd.ExcelWriter(exportName, engine='xlsxwriter')
      df.to_excel(writer, sheet_name='Sheet1', index=False)
      writer.save()
      print(f'dataframe successfully exported as: {exportName}')
      return df
    except:
      print('exportDataFrame function failed')

  # function to download and process data as specified above
  def exportAGOLdata(gis, itemID, exportName='exportXLS.xlsx'):
    try:
      DLdata = downloadReturn(gis, itemID)
      if str(DLdata[0])[-5] == '1':
        fullPath = DLdata[0]
      else: 
        fullPath = DLdata[1]
      print(fullPath)
      df = pd.read_csv(fullPath)
      display(df)
      dfED = exportDataFrame(df,exportName)
      print('<Dataframe Header created from AGOL item Export>')
      return dfED
    except:
      print(f'ERROR with {itemID} EXPORT')

  #Done 9-8-2020 - Just need to organize!
  # 9-9 -> Cleanup up and organized into functions, dfV2 local/global resolved
  def formatXLS(dfEX):
    # Lookup dict for site number linked to ID, define destination dataFrame
    siteList = {'DKC-ST-30':1,'DKC-ST-50':2,'FDP-SD-10':3,'ICD-ST-20':4,'ICD-ST-40':5,'ICD-ST-60':6,'KAR-ST-100':7,'KAR-ST-200':8,'NOT-ST-110':9,'NOT-ST-120':10,'NOT-ST-130':11,'NOT-ST-30':12,'NOT-ST-50':13,'NOT-ST-60':14,'NOT-ST-70':15,'NOT-ST-80':16,'NOT-ST-90':17,'PGC-ST-30':18,'PNC-ST-20':19,'PNC-ST-30':20,'PNC-ST-40':21,'PNC-ST-50':22,'PNC-ST-60':23,'PNC-ST-70':24,'QDP-LA-10':25,'RDP-SD-10':26,'SCD-ST-20':27,'SCD-ST-40':28,'SCD-ST-50':29,'SDP-LA-10':30,'UNT-ST-10':31,'UNT-ST-20':32,'ICD-ST-70':33,'HVC-ST-20':34,'NOT-ST-140':35,'NOT-ST-40':36,'DKC-ST-20':37,'DKC-ST-25':38,'NOT-ST-115':39,'PNC-ST-45':40,'ICD-ST-65':41,'SPC-ST-90':42,'SJR-ST-300':43}
    dfV2 = pd.DataFrame()
    # Define functions to transform/convert values for attributes
    def actType(QCval):
        if QCval == 'Field Measurement':
          return 'Field Msr/Obs'
        else:
          return 'Quality Control Field Replicate Msr/Obs'
    def FtoC(Ftemp):
      Ctemp = (int(Ftemp) - 32) * (5/9)
      return round(Ctemp, 2)
    def getDate(dateTime):
      splitDate = dateTime.split('/')
      if int(splitDate[0]) < 10:
        monthName = '0' + splitDate[0]
      else:
        monthName = splitDate[0]
      if int(splitDate[1]) < 10:
        dayName = '0' + splitDate[1]
      else:
        dayName = splitDate[1]
      splitDate = splitDate[2].split(' ')
      sampYear = splitDate[0]
      dateString = (f'{sampYear}{monthName}{dayName}')
      return dateString
    def isQC(qcInfo):
      returnVal = ''
      if qcInfo == 'Quality Control Sample Field Replicate':
        returnVal = ':QC'
      return returnVal
    def dateInfo(DATE_TIME):
      splitDate = DATE_TIME.split(' ')
      return splitDate
    def time24(DATE_TIME):
      splitTime = DATE_TIME.split(' ')
      dayTime = splitTime[1]
      dayTime = dayTime.split(':')
      hourTime = int(dayTime[0])
      minTime = int(dayTime[1])
      if minTime < 10:
        minTime = (f'0{str(minTime)}')
      if splitTime[2].upper() == 'PM':
        hourTime = int(dayTime[0]) + 12
      hourTime = hourTime - 5
      if hourTime < 10:
        hourTime = (f'0{str(hourTime)}')
      return hourTime,minTime
    def equipType(SAMP_EQUIP):
      if SAMP_EQUIP.upper() == 'YSI PRODSS':
        equipType = 'Probe/Sensor'
      else:
        equipType = 'Water Bottle'
      return equipType

    def convertValues():
      dfEX['SAMP_EQUIP'] = dfEX.apply(lambda row: equipType(row.SAMP_EQUIP), axis=1)
      dfEX['SITE #'] = dfEX.apply(lambda row: siteList[row.SITE_ID], axis=1)
      dfV2['SITE #'] = dfEX.apply(lambda row: siteList[row.SITE_ID], axis=1)
      numString = 'SITE #'
      dfV2['FIELD Activity ID'] = dfEX.apply(lambda row: row.SITE_ID + ':' + str(row['SITE #']) + ':'+ getDate(row.DATE_TIME)+':'+ str(time24(row.DATE_TIME)[0]) + str(time24(row.DATE_TIME)[1]) + isQC(row.QC), axis = 1)
      dfV2['Activity Type'] = dfEX.apply(lambda x: actType(x['QC']), axis = 1)
      dfV2['DATE'] = dfEX.apply(lambda row: dateInfo(row.DATE_TIME)[0], axis = 1)
      dfV2['TIME'] = dfEX.apply(lambda row: str(time24(row.DATE_TIME)[0])+':'+str(time24(row.DATE_TIME)[1]), axis = 1)
      dfV2['TEMP C'] = dfEX.apply(lambda x: FtoC(x['WATER_TEMP']), axis = 1)
      dfV2['SPCOND'] = dfEX['SPEC_COND_uS_cm']*.001
      dfV2['DO%'] = ''
      dfV2['pH'] = round(dfEX['pH'],1)
    def AssignCol(dfV2):
      dfCol = ['Monitoring Location','Sample Collection Equipment Name','TEMP F','TURB','DO mg/L','TOT. N mg/L','TOT. P mg/L','E. coli col/100 ml','NOTES']
      srcCol = ['SITE_ID','SAMP_EQUIP','WATER_TEMP','TURB_NTU','DO_mgl','TOT_NITRO_','TOT_PHOS_','Ecoli_100ml_','NOTES']
      x = 0
      for col in dfCol:
        dfV2[col] = dfEX[srcCol[x]]
        x+=1
      dfV2 = dfV2[['FIELD Activity ID','Monitoring Location','SITE #','Activity Type','Sample Collection Equipment Name','DATE','TIME','TEMP C','TEMP F','SPCOND','pH','TURB','DO%','DO mg/L','TOT. N mg/L','TOT. P mg/L','E. coli col/100 ml','NOTES']]
      return dfV2
    #function main body, convert values, name and organize DataFrame columns  
    convertValues()
    dfV3 = AssignCol(dfV2)
    return dfV3
  #--main function body flow--

  loginQuest = input(f'Do you need to Login to ArcGIS Online(YES) or (NO)? ')
  if loginQuest.upper() == 'YES':
    userLogin()
  searchQuest = input(f'ArcGIS Online Feature Service Search: ')
  searchReturn = searchByKeywords(gis, searchQuest)
  indexNumber = input(f'What is the index of the file search target? ')
  fileName = input(f'File Name for the XLSX export? ')
  exportName = (f'{fileName}.xlsx')
  dfEX = exportAGOLdata(gis, searchReturn[int(indexNumber)].id, exportName)
  dfExport = formatXLS(dfEX)
  exportDataFrame(dfExport, exportName)
  #display(dfExport)
  return dfExport

dataExport = exportWaterData()
dataExport.head()


In [ ]:
dfEZ = exportAGOLdata(gis, item[3].id, 'exportXLS.xlsx')

In [ ]:
dataExport = exportWaterData()
dataExport.head()

##working edited function to get 2019 data   
only one layer download, so function breaks when expecting water stations and water table data   
something adjust, check to see how many layers are part of the Feature Service Download

In [ ]:
  datTest = dataExport.dropna(subset=['WATER_TEMP'])
  datTest

In [ ]:
# this is the working section 9-21-2020
# Finally Looking good!!!

def downloadReturn(gisInfo, idString):
    try:
        downloadData = gisInfo.content.get(idString)
        dataPath = Path('/data')
        print(f'Downloading: {downloadData.title} to {dataPath} directory')
        if not dataPath.exists():
          dataPath.mkdir()
        # this portion for feature service
        downloadExport = downloadData.export(title=downloadData.title, export_format="CSV")
        zipPath = downloadExport.download(save_path=dataPath)
        # preparing to extract files to directory with item.title name
        extractPath = dataPath.joinpath(downloadData.title)
        # extract files to /data directory
        zipFiles = ZipFile(zipPath)
        zipFiles.extractall(path=extractPath)
        # delete CSV created by export
        print(f'Deleting CSV export generated by Download: {downloadExport.title} ID: {downloadExport.id}')
        itemToDelete = gisInfo.content.get(downloadExport.id)
        itemToDelete.delete()
        print(f'list of Files extracted to: {extractPath}')
        print(list(file.name for file in extractPath.glob('*')))
        returnArray = []
        for x in extractPath.glob('*'):
          returnArray.append(x)
        return returnArray
    except:
        print('ERROR DOWNLOAD did not workings!')

def exportAGOLdata(gis, itemID, exportName='exportXLS.xlsx'):
      DLdata = downloadReturn(gis, itemID)
      #if str(DLdata[0])[-5] == '1':
      #  fullPath = DLdata[0]
      #else: 
      #  fullPath = DLdata[1]
      # small change with 2019 data - only one layer in download
      fullPath = DLdata[0]
      print(fullPath)
      df = pd.read_csv(fullPath)
      #display(df)
      #dfED = exportDataFrame(df,exportName)
      print('<Dataframe Header created from AGOL item Export>')
      return df
      print(f'ERROR with {itemID} EXPORT')

# refactoring One more time 9-14.....
def formatXLS(dfSOURCE):
    # Lookup dict for site number linked to ID, define destination dataFrame
    siteList = {'DKC-ST-30':1,'DKC-ST-50':2,'FDP-SD-10':3,'ICD-ST-20':4,'ICD-ST-40':5,'ICD-ST-60':6,'KAR-ST-100':7,'KAR-ST-200':8,'NOT-ST-110':9,'NOT-ST-120':10,'NOT-ST-130':11,'NOT-ST-30':12,'NOT-ST-50':13,'NOT-ST-60':14,'NOT-ST-70':15,'NOT-ST-80':16,'NOT-ST-90':17,'PGC-ST-30':18,'PNC-ST-20':19,'PNC-ST-30':20,'PNC-ST-40':21,'PNC-ST-50':22,'PNC-ST-60':23,'PNC-ST-70':24,'QDP-LA-10':25,'RDP-SD-10':26,'SCD-ST-20':27,'SCD-ST-40':28,'SCD-ST-50':29,'SDP-LA-10':30,'UNT-ST-10':31,'UNT-ST-20':32,'ICD-ST-70':33,'HVC-ST-20':34,'NOT-ST-140':35,'NOT-ST-40':36,'DKC-ST-20':37,'DKC-ST-25':38,'NOT-ST-115':39,'PNC-ST-45':40,'ICD-ST-65':41,'SPC-ST-90':42,'SJR-ST-300':43}
  
    # Define functions to transform/convert values for attributes
    def actType(QCval):
        if QCval == 'Field Measurement':
          return 'Field Msr/Obs'
        else:
          return 'Quality Control Field Replicate Msr/Obs'
    def FtoC(Ftemp):
      # working here...
      if type(Ftemp) == 'int':
        Ctemp = (int(Ftemp) - 32) * (5/9)
        CtempRound = round(Ctemp, 2)
        return CtempRound
      else:
        return 'NULL'
    def getDate(dateTime):
      splitDate = dateTime.split('/')
      if int(splitDate[0]) < 10:
        monthName = '0' + splitDate[0]
      else:
        monthName = splitDate[0]
      if int(splitDate[1]) < 10:
        dayName = '0' + splitDate[1]
      else:
        dayName = splitDate[1]
      splitDate = splitDate[2].split(' ')
      sampYear = splitDate[0]
      dateString = (f'{sampYear}{monthName}{dayName}')
      return dateString
    def isQC(qcInfo):
      returnVal = ''
      if qcInfo == 'Quality Control Sample Field Replicate':
        returnVal = ':QC'
      return returnVal
    def dateInfo(DATE_TIME):
      splitDate = DATE_TIME.split(' ')
      return splitDate
    def time24(DATE_TIME):
      splitTime = DATE_TIME.split(' ')
      dayTime = splitTime[1]
      dayTime = dayTime.split(':')
      hourTime = int(dayTime[0])
      minTime = int(dayTime[1])
      if minTime < 10:
        minTime = (f'0{str(minTime)}')
      if splitTime[2].upper() == 'PM':
        hourTime = int(dayTime[0]) + 12
      hourTime = hourTime - 5
      if hourTime < 10:
        hourTime = (f'0{str(hourTime)}')
      return hourTime,minTime
    def equipType(SAMP_EQUIP):
      if SAMP_EQUIP.upper() == 'YSI PRODSS':
        equipType = 'Probe/Sensor'
      else:
        equipType = 'Water Bottle'
      return equipType

    def convertValuesNew(dfEY):
      dfV2 = pd.DataFrame()
      dfV2['SITE #'] = dfEY.apply(lambda row: siteList[row.SITE_ID], axis=1)
      numString = 'SITE #'
      dfV2['FIELD Activity ID'] = dfEY.apply(lambda row: row.SITE_ID + ':' + str(row['SITE #']) + ':'+ getDate(row.DATE_TIME)+':'+ str(time24(row.DATE_TIME)[0]) + str(time24(row.DATE_TIME)[1]) + isQC(row.QC), axis = 1)
      dfV2['Activity Type'] = dfEY.apply(lambda x: actType(x['QC']), axis = 1)
      dfV2['DATE'] = dfEY.apply(lambda row: dateInfo(row.DATE_TIME)[0], axis = 1)
      dfV2['TIME'] = dfEY.apply(lambda row: str(time24(row.DATE_TIME)[0])+':'+str(time24(row.DATE_TIME)[1]), axis = 1)
      dfV2['TEMP C'] = dfEY.apply(lambda x: FtoC(x['WATER_TEMP']), axis = 1)
      dfV2['SPCOND'] = dfEY['SPEC_COND_uS_cm']*.001
      dfV2['DO%'] = ''
      dfV2['pH'] = round(dfEY['pH'],1)
      return dfV2

#    def AssignCol(dfV3, dfEX):
#      dfCol = ['Monitoring Location','Sample Collection Equipment Name','TEMP F','TURB','DO mg/L','TOT. N mg/L','TOT. P mg/L','E. coli col/100 ml','NOTES']
#      srcCol = ['SITE_ID','SAMP_EQUIP','WATER_TEMP','TURB_NTU','DO_mgl','TOT_NITRO_','TOT_PHOS_','Ecoli_100ml_','NOTES']
#      x = 0
#      for col in dfCol:
#        dfV3[col] = dfEX[srcCol[x]]
#        x+=1
#      dfV3 = dfV3[['FIELD Activity ID','Monitoring Location','SITE #','Activity Type','Sample Collection Equipment Name','DATE','TIME','TEMP C','TEMP F','SPCOND','pH','TURB','DO%','DO mg/L','TOT. N mg/L','TOT. P mg/L','E. coli col/100 ml','NOTES']]
#      return dfV3
    dfNewSource = dfSOURCE.dropna(subset=['WATER_TEMP'])
    
    #dfSOURCE['SAMP_EQUIP'] = dfSOURCE.apply(lambda row: equipType(row.SAMP_EQUIP), axis=1)
    #dfSOURCE['SITE #'] = dfSOURCE.apply(lambda row: siteList[row.SITE_ID], axis=1)
    dfNewSource['SAMP_EQUIP'] = dfNewSource.apply(lambda row: equipType(row.SAMP_EQUIP), axis=1)
    dfNewSource['SITE #'] = dfNewSource.apply(lambda row: siteList[row.SITE_ID], axis=1)

    dfNewMod = convertValuesNew(dfNewSource)

    dfCol = ['Monitoring Location','Sample Collection Equipment Name','TEMP F','TURB','DO mg/L','TOT. N mg/L','TOT. P mg/L','E. coli col/100 ml','NOTES']
    srcCol = ['SITE_ID','SAMP_EQUIP','WATER_TEMP','TURB_NTU','DO_mgl','TOT_NITRO','TOT_PHOS','Ecoli_100ml','NOTES']
    x = 0
    #display(dfSOURCE.head())
    for col in dfCol:
      dfNewMod[col] = dfNewSource[srcCol[x]]
      x+=1
    
    dfFINAL = dfNewMod[['FIELD Activity ID','Monitoring Location','SITE #','Activity Type','Sample Collection Equipment Name','DATE','TIME','TEMP C','TEMP F','SPCOND','pH','TURB','DO%','DO mg/L','TOT. N mg/L','TOT. P mg/L','E. coli col/100 ml','NOTES']]
    
    #dfEZ = dfEX.dropna(subset=['WATER_TEMP'])

    #dfSourceConv = convertValuesSource(dfSOURCE)
    #dfNewMod = convertValuesNew(dfSOURCE)
    #dfFINAL = AssignCol(dfNewMod, dfSOURCE)
    #return dfNewMod
    writer = pd.ExcelWriter('NHBP-2019-DATA.xlsx', engine='xlsxwriter')
    dfFINAL.to_excel(writer, sheet_name='Sheet1', index=False)
    writer.save()
    print(f'dataframe successfully exported as: NHBP-2019-DATA.xlsx')
    
    return dfFINAL


dfExport = exportAGOLdata(gis, searchReturn[0].id, exportName='exportXLS.xlsx')
#dfExport.head()
dfFormat = formatXLS(dfExport)
#dfFormat.head()

Downloading: WATER_DATA_2019_CLEANED to /data directory
Deleting CSV export generated by Download: WATER_DATA_2019_CLEANED ID: 97f2fc191e81450c8d357172c0683197
list of Files extracted to: /data/WATER_DATA_2019_CLEANED
['WATER_DATA_2019_CLEANED_0.csv']
/data/WATER_DATA_2019_CLEANED/WATER_DATA_2019_CLEANED_0.csv
<Dataframe Header created from AGOL item Export>
dataframe successfully exported as: NHBP-2019-DATA.xlsx


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Done and downloaded 9-21-2020 ##
Working 9-21 on 2019 data export   
Not sure where I was last working, may need to review the entire workflow   
May take several of the features out of the functio to simplify

In [ ]:
dfFormat.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,23,24,25,26,27,28,29,30,31,32,33,34,35,36
FIELD Activity ID,PNC-ST-40:21:20190725:1455,UNT-ST-10:31:20190725:1510:QC,UNT-ST-10:31:20190725:1509,PNC-ST-50:22:20190725:1526,NOT-ST-120:10:20190815:1319,NOT-ST-50:13:20190815:1401,NOT-ST-110:9:20190815:1352,NOT-ST-70:15:20190815:1402,PNC-ST-30:20:20190815:1426,PNC-ST-40:21:20190827:1219,PNC-ST-40:21:20190827:1220:QC,PNC-ST-45:40:20190827:1238,PNC-ST-50:22:20190827:1249,PNC-ST-60:23:20190827:1312,NOT-ST-110:9:20190827:1500,NOT-ST-80:16:20190828:1350,NOT-ST-90:17:20190828:1411,NOT-ST-110:9:20190828:1441,SCD-ST-40:28:20190710:1312,PNC-ST-60:23:20190710:1325,PNC-ST-60:23:20190710:1326:QC,PNC-ST-50:22:20190710:1336,ICD-ST-40:5:20190710:1347,PNC-ST-60:23:20190719:0940,PNC-ST-60:23:20190719:0941:QC,PNC-ST-50:22:20190720:0-355,NOT-ST-110:9:20190720:0-245,ICD-ST-40:5:20190930:1058,PNC-ST-50:22:20190930:1108,PNC-ST-60:23:20190930:1118,SCD-ST-40:28:20190930:1126,SCD-ST-40:28:20190930:1130:QC
Monitoring Location,PNC-ST-40,UNT-ST-10,UNT-ST-10,PNC-ST-50,NOT-ST-120,NOT-ST-50,NOT-ST-110,NOT-ST-70,PNC-ST-30,PNC-ST-40,PNC-ST-40,PNC-ST-45,PNC-ST-50,PNC-ST-60,NOT-ST-110,NOT-ST-80,NOT-ST-90,NOT-ST-110,SCD-ST-40,PNC-ST-60,PNC-ST-60,PNC-ST-50,ICD-ST-40,PNC-ST-60,PNC-ST-60,PNC-ST-50,NOT-ST-110,ICD-ST-40,PNC-ST-50,PNC-ST-60,SCD-ST-40,SCD-ST-40
SITE #,21,31,31,22,10,13,9,15,20,21,21,40,22,23,9,16,17,9,28,23,23,22,5,23,23,22,9,5,22,23,28,28
Activity Type,Field Msr/Obs,Quality Control Field Replicate Msr/Obs,Field Msr/Obs,Field Msr/Obs,Field Msr/Obs,Field Msr/Obs,Field Msr/Obs,Field Msr/Obs,Field Msr/Obs,Field Msr/Obs,Quality Control Field Replicate Msr/Obs,Field Msr/Obs,Field Msr/Obs,Field Msr/Obs,Field Msr/Obs,Field Msr/Obs,Field Msr/Obs,Field Msr/Obs,Field Msr/Obs,Field Msr/Obs,Quality Control Field Replicate Msr/Obs,Field Msr/Obs,Field Msr/Obs,Field Msr/Obs,Quality Control Field Replicate Msr/Obs,Field Msr/Obs,Field Msr/Obs,Field Msr/Obs,Field Msr/Obs,Field Msr/Obs,Field Msr/Obs,Quality Control Field Replicate Msr/Obs
Sample Collection Equipment Name,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor,Probe/Sensor
DATE,7/25/2019,7/25/2019,7/25/2019,7/25/2019,8/15/2019,8/15/2019,8/15/2019,8/15/2019,8/15/2019,8/27/2019,8/27/2019,8/27/2019,8/27/2019,8/27/2019,8/27/2019,8/28/2019,8/28/2019,8/28/2019,7/10/2019,7/10/2019,7/10/2019,7/10/2019,7/10/2019,7/19/2019,7/19/2019,7/20/2019,7/20/2019,9/30/2019,9/30/2019,9/30/2019,9/30/2019,9/30/2019
TIME,14:55,15:10,15:09,15:26,13:19,14:01,13:52,14:02,14:26,12:19,12:20,12:38,12:49,13:12,15:00,13:50,14:11,14:41,13:12,13:25,13:26,13:36,13:47,09:40,09:41,0-3:55,0-2:45,10:58,11:08,11:18,11:26,11:30
TEMP C,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
TEMP F,70,76.6,76.6,71.1,70.7,65.9,69.6,68.8,69.6,65.9,65.8,66,66.5,66.5,67.5,66.5,66.6,66.8,67.5,74.6,76.1,74,70.3,76.8,76.8,76.5,72.7,62.2,61.2,61.7,62.1,62.1
SPCOND,0.4665,0.4825,0.4829,0.4622,0.561,0.584,0.562,0.561,0.527,0.506,0.504,0.504,0.504,0.502,0.558,0.557,0.558,0.558,0.549,0.4032,0.4373,0.4034,0.534,0.4688,0.4675,0.4684,0.562,0.3463,0.4085,0.4112,0.3659,0.3659


In [ ]:
userLogin()
searchReturn = searchByKeywords(gis, 'water data cleaned 2019')
searchReturn[0].id

ArcGIS Online USER ID: ekerney_nhbptribe
PASSWORD: ··········
SUCCESS - CONNECTED TO: ekerney_nhbptribe ACCOUNT as <gis>
GIS @ https://nhbptribe.maps.arcgis.com version:8.2
Additional User Login(YES/NO)? no
YOU MAY NOW PROCEED...
0 - WATER_DATA_2019_CLEANED - 00a27de719a8420c82a6d2985a8ff735 - Feature Service - ['/Categories/ENVIRONMENTAL DATA/WATER QUALITY'] - 2020-09-13 12:55:55
1 - WATER_DATA_2019_CLEANED - 7410847228994c89885b60a0ede9c91c - CSV - ['/Categories/ENVIRONMENTAL DATA/WATER QUALITY'] - 2020-09-13 12:55:43
2 - WATER_DATA_2019_CLEANED - 2937e8617ba648bca631af9a8978ce3e - CSV Collection - [] - 2020-09-21 19:12:06
3 - WATER_DATA_2019_CLEANED - 929abf1f25e14edca449f5f526bffbe5 - CSV Collection - [] - 2020-09-21 17:22:01
4 - WATER_DATA_2019_CLEANED - 95dea90331e94ac1b79850820d20b089 - CSV Collection - [] - 2020-09-21 19:29:14


'00a27de719a8420c82a6d2985a8ff735'

In [ ]:
testString = 'yes its stringy'
testInt = 42

type(testString)

str

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving WATER_DATA_2019_CLEANED.csv to WATER_DATA_2019_CLEANED.csv


In [ ]:
dataPath = '/data/WATER_DATA_2019_CLEANED/WATER_DATA_2019_CLEANED_0.csv'
df = pd.read_csv(dataPath)
df

In [ ]:
df.isnull().sum()

In [ ]:
delList = searchByKeywords(gis,'CSV-collection')
delMultiple(gis,delList)
#delList[17]